In [1]:
"""PROGRAM TO COMBINE METAGENOMES FROM ALL SAMPLE SITES INTO A SINGLE DATABASE AS A 
LIST OF LISTS.  EACH TAXON HSS A ROW: [ TAXON, SUM OVER ALL SITES, AVERAGE FREQ ] 
ONLY TAXA WITH AVERAGE FREQUENCIES > 10^-5 ARE INCLUDED IN THE OUTPUT FILE.
"""
numprocess = 410
import math
from google.colab import drive
drive.mount('/content/gdrive')

#1. THE OUTFILE NAME. IF A MG IS TO LEFT LEFT OUT,INSERT IT IN PLACE OF THE "0":
outname = '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Frqs.All.MGs.Joined.csv'

#2. PASTE IN THE NAMES OF ALL THE INPUT METAGENOME FILES: 
files = ['/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site1.Loop1.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site2.Loop3.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site3.Loop2.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site4.Loop1.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site5.Loop3.TRANSPOSED.csv',
         '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Site6.Loop2.TRANSPOSED.csv']
nummetas =  len(files)

stringmetas =[]
for x in range(nummetas):   # for reading files and converting each taxon to a list of [taxon, number]
  with open(files[x], 'r') as f:
    data = f.read()
    #print(repr(data)[:100])
    stringmetas.append(str(data))
  f.close()
print('Number of metagenomes =', len(stringmetas))

cleanmetas =[]            # gets rid of unwanted test, converts number from string to integer
for x in range(nummetas):
  datastring = stringmetas[x].replace('unclassified (derived from ', '').replace(')', '').replace('Candidatus ', '')
  metalines = datastring.split('\n')
  #print(metalines)
  metalines.pop()
  integermeta =[]
  for y in range(len(metalines)):
    row = metalines[y].split(',')
    integermeta.append([row[0], int(row[1]), 0.0])
  cleanmetas.append(integermeta)

totalslist =[]             # counts the total number of taxa in each metagenome
for x in range(nummetas):
  count =0
  for y in range(len(cleanmetas[x])):
    count += cleanmetas[x][y][1]
  totalslist.append(count)
print('Total counts for each metagenone: ', totalslist)

for x in range(nummetas):
  for y in range(len(cleanmetas[x])):
    cleanmetas[x][y][2] += cleanmetas[x][y][1] / totalslist[x]

taxonset = set()
for x in range(nummetas):
  for y in range(numprocess):
    if cleanmetas[x][y][0] not in taxonset:
      taxonset.add(cleanmetas[x][y][0])

taxonlist = list(taxonset)
cvcm =[]
for x in range(len(taxonlist)):
  row = [ taxonlist[x], 0.0 , 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  for y in range(nummetas):
    for z in range(len(cleanmetas[y])):
      if cleanmetas[y][z][0] == taxonlist[x]:
        row[1] += cleanmetas[y][z][2]
  cvcm.append(row)

cvcm.sort(key = lambda x: x[1])
cvcm.reverse()

for x in range(len(cvcm)):
  for y in range(nummetas):
    for z in range(len(cleanmetas[y])):
      if cleanmetas[y][z][0] == cvcm[x][0]:
        cvcm[x][y+2] = cleanmetas[y][z][2]

with open(outname, 'w') as g:
 for x in range(len(cvcm)):
   outline = cvcm[x][0] +','+ str(cvcm[x][2]) +','+ str(cvcm[x][3]) +','+ str(cvcm[x][4]) +','+ str(cvcm[x][5]) +','+ str(cvcm[x][6]) +','+ str(cvcm[x][7])
   print(outline)
   g.write(outline +'\n')
g.close()
print(len(cvcm), 'Taxa.  Done!')


Mounted at /content/gdrive
Number of metagenomes = 6
Total counts for each metagenone:  [38228, 18354, 21353, 11919, 18447, 17611]
Gammaproteobacteria,0.04415611593596317,0.09294976571864444,0.0821898562262914,0.07114690829767598,0.10484089553856996,0.08131281585372778
Beggiatoa,0.0979910013602595,0.016399694889397406,0.02926989181847984,0.005453477640741673,0.0034151894617010896,0.005110442337175629
Deltaproteobacteria,0.0335617871717066,0.021630162362427808,0.027443450569006697,0.03070727410017619,0.01154659294194178,0.023905513599454885
Desulfobacterium,0.033823375536256146,0.016672115070284408,0.01784292605254531,0.023324104371172077,0.011112918089662275,0.019362898188632106
Nitrosopumilus,0.0035314429214188554,0.01885147651738041,0.01264459326558329,0.009984059065357832,0.05420935653493793,0.010675146215433536
Bacteroides,0.019880715705765408,0.015255530129672006,0.014143211726689458,0.01602483429817938,0.0094324280370792,0.016296632786326728
Desulfatibacillum,0.02186878727634195,